In [2]:
import pandas as pd
import numpy as np
import copy
# elo_df = pd.read_csv("Current ELO.csv")

import tkinter as tk
from tkinter import ttk # Submodule for additional widgets
from tkinter.messagebox import showinfo
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import asksaveasfilename

# WORKS PERFECTLY

# This keeps track of the names of each entry widget - we'll need this to get the names of
# each thing entered into the 
# NOTE: Obsolete - I have figured out a better way to keep track of widgets. 
# Keep - still the best way to add stuff via csv
entries = []

# Not actually necessary - just used to keep the frames in the right order.
num_player = 0
error_frame = None # Required for error handling
previous_match_history = None

def add_player():
    """
    This function is tied to a button that adds a frame containing a label and a entry widget.
    
    These widgets are created in a frame above the Add player button
    """
    # Allows us to edit the entries as well as change the number of players
    global entries
    global num_player
    global error_frame
    
    # If there is an error_frame instance, destroy it when adding a new player and reinitialize
    # it as None
    if error_frame is not None:
        error_frame.destroy()
        error_frame = None
    
    # Add 1 to the number of players - helps with organizing the rows
    num_player += 1
    
    # Create the frame
    # Note that the name of a widget CANNOT start with an uppercase letter
    # name = f"frame {num_player}"
    frame = tk.Frame(input_frame, borderwidth=2, relief="groove")
    
    # Put the frame in row X, where X is the number of players.
    # The add players button and the get player_names button will also be shifted downwards
    frame.grid(row = num_player, column = 0)
    add_button.grid(row = num_player + 2, column = 0, pady = 5)
    get_button.grid(row = num_player + 3, column = 0, pady = 10)
    
    # Create the label, entry, and remove player widgets
    # Note that text was originally f"Player {num_player}", but I couldn't get it to 
    # work with frame.destroy
    label = tk.Label(frame, text = f"Player Name")
    entry = tk.Entry(frame)
    
    # Can't get remove_player to work - removing players will mess up the order of the 
    # entries in the entries list. name = f"button {num_player}" - FIXED
    
    # Button that destroys the frame it's in.
    button = tk.Button(frame, text = "Remove Player", command = frame.destroy)
    
    # Append the entry instance to a list - we'll need this to get the names out - OBSOLETE
    entries.append(entry)
    
    # The position of the label/entry in the created frame
    label.grid(row = 0, column = 0)
    entry.grid(row = 0, column = 1)
    button.grid(row = 0, column = 2)

    
# Create the list of players
list_of_players = list()

def get_player_names():
    """
    Function that collects all the text written in each entry and appends it to player_list
    
    This is currently bound to a button press.
    """
    global list_of_players
    global error_frame
    global input_frame
    
    # Print list of frames in the input_frame
    list_of_frames = [i for i in input_frame.winfo_children() if i.winfo_class() == "Frame"]
    
    list_of_entries = list()
    
    # For each frame, retrieve the entry containing the name of the player
    # With this, no need to keep track of entry instances in list
    for frame in list_of_frames:
        # Each frame has multiple children - Label, Entry, Button
        for child in frame.winfo_children():
            # We want the entry - append that to a list
            if child.winfo_class() == "Entry":
                list_of_entries.append(child)
    
    # print(list_of_entries)
    
    # Create an error_frame if there are too few players.
    if len(list_of_entries) < 3:
        error_frame = tk.Frame(input_frame)
        error_frame.grid(row = num_player + 1, column = 0, pady = 10)
        error_label = tk.Label(error_frame, text = "Error: A Swiss Tournament cannot be run\nwith less than three players.\nPlease add another player.", fg = "red")
        error_label.grid(row = 0, column = 0)
        return
    
    # list_of_entries is a list of the current entry instances
    for entry in list_of_entries:
        list_of_players.append(entry.get())
    
    # This closes the window after getting the player names. 
    # Do we want this? Or would it be better to destroy the frames and create
    # new frames?
    window.destroy()

options_error_frame = None

def browse_csv():
    """
    Function that automatically adds player names to the list.
    
    Works great!
    """
    global options_frame
    global options_error_frame
    
    filepath = askopenfilename(
        filetypes=[("CSV Files", "*.csv"), ("All Files", "*.*")]
    )
    
    # If no file selected, do nothing
    if not filepath:
        return
    
    temp_df = pd.read_csv(filepath)
    
    # Need to do error handling for this step
    try:
        for player in temp_df["Player"]:
            add_player()
            
            
            entries[-1].insert(0, player)
        
        # Get rid of any error messages if the file is correctly produced
        if options_error_frame is not None:
            options_error_frame.destroy()
            options_error_frame = None
            
    except:
        # Create an error message if there is an error.
        options_error_frame = tk.Frame(options_frame)
        options_error_frame.grid(row = 3, column = 0, pady = 10)
        options_error_label = tk.Label(options_error_frame, text = "Error: The file did not\ncontain a list of players.\n\n"
                                       "Check that the column is\ncorrectly labelled 'Player'", fg = "red")
        options_error_label.grid(row = 0, column = 0)
        return

    
# OBSOLETE
# def remove_player(event):
#     # str(...) gives me the name of the widget
#     button_name = str(event.widget).split(".")[-1]
    
#     frame_name = "frame" + button_name[-1]
    
#     input_frame.nametowidget.destroy()
#     # See if i can get all instances of entry from input_frame
#     # Yup this works
#     # print([i for i in input_frame.winfo_children() if i.winfo_class() == "Frame"])

def get_match_history():
    global previous_match_history
    global options_error_frame
    global options_frame
    
    filepath = askopenfilename(
        filetypes=[("CSV Files", "*.csv"), ("All Files", "*.*")]
    )
    
    # If no file selected, do nothing
    if not filepath:
        return
    
    temp_df = pd.read_csv(filepath, index_col = 0)
    
    starting_player_list = list(set(temp_df["Player_1"]).union(set(temp_df["Player_2"])))
    
    # Remove Byes if any
    if "Bye" in starting_player_list:
        starting_player_list.pop(starting_player_list.index("Bye"))
        
    # Find all players in the tournament by looking through player 1 and player 2
    try:
        for player in starting_player_list:
            add_player()

            # Automatically insert names into left side frame
            entries[-1].insert(0, player)

            # Get rid of any error messages if the file is correctly produced
            if options_error_frame is not None:
                options_error_frame.destroy()
                options_error_frame = None
    except Exception as e:
        # Note that this is how you print your error messages. Very useful.
        # print(e)
        # Create an error message if there is an error.
        options_error_frame = tk.Frame(options_frame)
        options_error_frame.grid(row = 3, column = 0, pady = 10)
        options_error_label = tk.Label(options_error_frame, text = "Error: The file did not\ncontain a list of players.\n\n"
                                       "Check that the column is\ncorrectly labelled 'Player'", fg = "red")
        options_error_label.grid(row = 0, column = 0)
        return
    
    previous_match_history = temp_df.transpose().to_dict()
    
############################################################################################    

# Note that the below can't be put into a function unless you call global window - I believe.
window = tk.Tk()
window.title("Player List")

# Adapts to height and width
window.rowconfigure(0, minsize = 10, weight = 1)
window.columnconfigure([0,1], minsize = 150, weight = 1)

# Add a secondary frame with additional options
options_frame = tk.Frame(window, relief = tk.RAISED, borderwidth = 2)
options_frame.grid(row = 1, column = 1)

# Give the option to supply a player_list from a .csv file
retrieve_frame = tk.Frame(options_frame, relief = tk.GROOVE, borderwidth = 1)
retrieve_frame.grid(row = 0, column = 0, sticky = "nsew")
retrieve_frame.columnconfigure(0, minsize = 250)
retrieve_label = tk.Label(retrieve_frame, text = "You can choose to retrieve player\n"
                          "names from a .csv file.\n\nAll player names should be in a\n"
                          "column called 'Player'.")
retrieve_button = tk.Button(retrieve_frame, text = "Browse", command = browse_csv)
retrieve_label.grid(row = 0, column = 0)
retrieve_button.grid(row = 0, column = 1, padx = 15, pady = 50)

# Allow user to continue from previous tournament.
continue_frame = tk.Frame(options_frame, relief = tk.GROOVE, borderwidth = 1)
continue_frame.grid(row = 1, column = 0, sticky = "nsew")
continue_frame.columnconfigure(0, minsize = 250)
continue_label = tk.Label(continue_frame, text = "Continue a previous\ntournament.\n\n"
                          "Recommend only using\nmatch histories created\nby this program.")
continue_button = tk.Button(continue_frame, text = "Browse", command = get_match_history)
continue_label.grid(row = 0, column = 0)
continue_button.grid(row = 0, column = 1, padx = 15, pady = 50)

# Give the option to provide a seeding before the tournament starts
# If unticked, then no seeding will be used.
seeding_frame = tk.Frame(options_frame, relief = tk.GROOVE, borderwidth = 1)
seeding_frame.grid(row = 2, column = 0, sticky = "nsew")
var1 = tk.IntVar()
seeding_checkbox = tk.Checkbutton(seeding_frame, text = "Use seeding (Players are seeded in order entered)", 
                                  justify = tk.LEFT,
                                  variable = var1)
seeding_checkbox.grid(row = 0, column = 0, columnspan = 2)

# Add the original add player and get player names buttons
# We put these into their own frame
input_frame = tk.Frame(window, borderwidth = 2)
input_frame.grid(row = 1, column = 0, sticky = "nsew")
input_frame.columnconfigure(0, minsize = 100)

add_button = tk.Button(input_frame, text = "Add Player", command = add_player)
add_button.grid(row = 2, column = 0, pady = 5, padx = 20)
get_button = tk.Button(input_frame, text = "Begin Tournament", command = get_player_names)
get_button.grid(row = 3, column = 0, pady = 10, padx = 20)


window.mainloop()

In [3]:
# Explain later if this works

def dict_symmetric_difference(a, b):
    return {k: a[k] if k in a else b[k] for k in  # break here to fit without scrolling
            set(a.keys()).symmetric_difference(b.keys())}

In [4]:
def has_player_had_a_bye(player, match_history):
    """
    Looks through match_history to see if the player has had a bye
    
    returns Boolean
    """
    
    for match in match_history:
        if player == match_history[match]["Player_1"]:
            if match_history[match]["Player_2"] == "Bye":
                return True
        elif player == match_history[match]["Player_2"]:
            if match_history[match]["Player_2"] == "Bye":
                return True
            
    return False
            
# has_player_had_a_bye("James", match_history)

def determine_pairings(standings_df, match_history, seeding = False):
    """
    Determines pairings for a given standings
    
    standings_df should have:
        "Player", "Points", "OMW", "ELO"
        
    There appears to be a bug where if the algorithm cannot find a successful initial
    match, the entire algorithm breaks down <- I suspect it has something to do with
    the disallowed pairings dictionary.
    """
    global num_rounds
    # Create a list to hold our pairings
    pairings = list()
    
    # Sort the standings_df from best performing to worst performing
    # Note that we use sample(frac = 1) to ensure that the duplicates are not sorted
    # alphabetically
    if num_rounds == 1 and seeding:
        sorted_df = standings_df.sort_values(["Points", "OMW"], ascending = False)
    else:
        sorted_df = standings_df.sample(frac = 1).sort_values(["Points", "OMW"], ascending = False)
    
    # Note that this preserves the ordering
    player_list = list(sorted_df["Player"].unique())
    
    pristine_list = copy.deepcopy(player_list)
    
    # print(player_list)
    
    # If there odd number of players, the worst performing player has a bye
    if len(player_list) % 2 != 0:
        
        # Iterate backwards starting from the worst player
        for player in player_list[-1::-1]:
        
            # We need to check if the worst performing player already had a bye. 
            # In which case they cannot have a bye again.
            if has_player_had_a_bye(player, match_history):
                continue
                
            # If the player doesn't have a bye, then pop that player from the list
            # and give them a bye
            else:
                pairings.append([player_list.pop(player_list.index(player)), "Bye"])
                break
    
    # At this point, there should be an even number of players
    # Create a clean copy of the current list
    pristine_list = copy.deepcopy(player_list)
    
    # Create a dictionary for disallowed pairings - it will be a dictionary
    # with player names as keys and empty lists as values
    # This is for certain situations where two players can't play because they would
    # cause an illegal pairing later down the line
    
    # disallowed_pairings = {}
    
    disallowed_pairings = {i: {} for i in range(2, len(player_list) + 1, 2)}
    
    for grouping in disallowed_pairings:
        for player in player_list:
            disallowed_pairings[grouping][player] = list()
        
    # Code runs until player_list is empty, i.e. there are no players left to be paired
    while len(player_list) > 0:
        
        # We are going to try and match the first player in player_list - the "primary"
        # player. They'll be denoted by player_list[0]
        
        # First check - who has this player played before?
        previously_played = []
        
        for match in match_history:
        
            # If our primary player was one of the belligerents, put the other guy in
            # previously played
            if player_list[0] == match_history[match]["Player_1"]:
                previously_played.append(match_history[match]["Player_2"])
            elif player_list[0] == match_history[match]["Player_2"]:
                previously_played.append(match_history[match]["Player_1"])
                
        # Add any disallowed pairings for the primary player
        previously_played.extend(disallowed_pairings[len(player_list)][player_list[0]])
        print(player_list)
        print("matching", player_list[0])
        print(previously_played)
        print()
        
        # If the current player has played ALL remaining players
        # NOTE: This may not be stable in this build - try a few times.
        if set(player_list[1:]).issubset(previously_played):
            # Then the previous pairing cannot be allowed. 
            # Remove the previous pairing, and add it back to FRONT of the player_list
            try:
                # Right, THIS seems to be causing issues.
                # Specifically, it appears to be mixing up the order of the player_list
                # Ahh i see what's happening - when you try a second choice pairing and
                # it fails, the pairing is put back into player_list, but in the 
                # second choice order, which of course fucks up the order
                # in which the program attempts the next pairing. That makes sense.
                
                # Right, this is objectively wrong and can lead to some slightly weird
                # matchups near the end. 
                # However, it should no longer outright fail anymore
                temp = pairings.pop()
                temp.extend(player_list)
                player_list = temp
                
#                 wanted_length = len(player_list) + 2
                
#                 # At this point, we need to go back to the original list
#                 player_list = copy.deepcopy(pristine_list[-wanted_length:])
#                 print(player_list)
                
                # Then ensure that the previous pairing cannot be allowed by setting 
                # those players to have illegal pairings
                # Was originally disallowed_pairings[player_list[0]].player_list(temp[1])
                disallowed_pairings[len(player_list)][temp[0]].append(temp[1])
                print(disallowed_pairings)
                print()
                
                # This STILL causes problems, unfortunately - 
                # basically if we are trying 4 -> 2 and nothing works, we go back up to 6...
                # but we should wipe the disallowed pairings dictionary that we made for fixing
                # 4 -> 2
                
                # This line causes problems down the line I think. I haven't confirmed this
                # If there are any bugs, we'll need to find them.
                # NOTE: There still DO seem to be some bugs. 
                # disallowed_pairings[player_list[1]].append(player_list[0])

                # We want to skip the pairing step, because now our "primary" player has 
                # changed. Go back to start of loop
                continue
            except Exception as e:
                # Print error logs to try and find out what is happening
                print(e)
#                 print(player_list)
#                 print(match_history)
#                 print(standings_df)
#                 print(disallowed_pairings)
                return
        
        # Pair with next highest legal player. 
        for index in range(1, len(player_list)):
            # If the players have not played before, add them to pairings and remove them
            # from the player_list
            # Then break the for loop.
            if player_list[index] not in previously_played:
                # Note that we have to use .pop(index-1) because
                # .pop(0) happens FIRST, so all indices are moved back by 1.
                pairings.append([player_list.pop(0), player_list.pop(index-1)])
                
                break
    
    # Return our list of pairings
    return pairings

def update_standings(match_history,
                     list_of_remaining_players, 
                     sort_by_elo = False):
    """
    Generates the standings of a given match history dictionary.
    
    Requires match_history plus the list of remaining players (in case of drops)
        Also because its easier.
        
    sort_by_elo - set to False by default
        
    Also give the list of ELOs - generally only used for sorting.
    
    Might be slow for massive numbers of players, but honestly should be okay even for 
    hundreds of players, which is what is being asked for
    
    The returned standings_df has the following:
        "Player", "Points", "OMW", "ELO"
    """
    
    # We have an opponents key, but we'll get rid of it after calculating OMW
    standings_dict = {
        "Player": list_of_remaining_players,
        "Played": [],
        "Points": [],
        "OMW": [],
        "Score For": [],
        "Score Against": [],
        "Opponents": []
    }
    
    # For each player 
    for player in list_of_remaining_players:
        
        played = 0
        points = 0
        score_for = 0
        score_against = 0
        opponents = []
        
        # Check to see if they were involved in a match
        for match in match_history:
            # If they were player 1
            if player == match_history[match]["Player_1"]:
                # Increased played by 1
                played += 1
                # Fill the opponents list with the names of each opponent
                opponents.append(match_history[match]["Player_2"])
                
                # Update the scores for and against that player
                score_for += float(match_history[match]["Score"].split("-")[0])
                score_against += float(match_history[match]["Score"].split("-")[1])
                
                # If they won, add 1 point
                if match_history[match]["Result"] == "W":
                    points += 1
                # If they drew, add 0.5 point
                elif match_history[match]["Result"] == "D":
                    points += 0.5
            
            # If they were player 2
            elif player == match_history[match]["Player_2"]:
                played += 1
                
                # Update the scores for and against that player
                # Note that if they are player 2
                score_for += float(match_history[match]["Score"].split("-")[1])
                score_against += float(match_history[match]["Score"].split("-")[0])
                
                opponents.append(match_history[match]["Player_1"])
                
                # If they drew, add 0.5 point
                if match_history[match]["Result"] == "D":
                    points += 0.5
                    
        # Once we've looped through all the matches
        # Append the stats to the standings_dict
        standings_dict["Played"].append(played)
        standings_dict["Points"].append(points)
        standings_dict["Score For"].append(score_for)
        standings_dict["Score Against"].append(score_against)
        standings_dict["Opponents"].append(opponents)
    
    # We can only calculate OMW after all matches have been added to the standings
    for opponent_list in standings_dict["Opponents"]:
        
        running_omw = []
        
        # For each opponent they've previously played, find their win percentage.
        for opponent in opponent_list:
            
            try:
                # Find index of their opponent
                opponent_index = standings_dict["Player"].index(opponent)

                running_omw.append(standings_dict["Points"][opponent_index] / 
                                   standings_dict["Played"][opponent_index])
                
            # If the opponent was a "bye" (i.e., they're not in the standings_dict)
            # They count as an opponent with 0% wins
            except:
                running_omw.append(0)
        
        if len(running_omw) == 0:
            standings_dict["OMW"].append(0)
        else:
            standings_dict["OMW"].append(np.round(np.mean(running_omw), 3))
    
    # Remove the opponents key:value pair
    standings_dict.pop("Opponents")
    
    # Turn the dictionary into a dataframe
    standings_df = pd.DataFrame.from_dict(standings_dict).sort_values(["Points", "OMW", "Score For", "Score Against"], ascending = False)
    
    return standings_df

def add_result(match_num, player_one, player_two, player_one_score, player_two_score, match_history):
    """
    Prompts user for result and records it in the given dictionary
    
    Arguments:
    
    match_num
    
    player_one - string
    player_two - string
    player_one_score
    player_two_score
    
    match_history - dictionary with the match numbers as keys to a dictionary
        "Player_1" 
        "P"
        "Result" 
        with empty lists as values.
    
    Does not return anything, merely updates the match_history dictionary with the
    results
    """
    
    
    # Note that the "Result" is from the perspective of Player_1 - it should either be
    # W or D
    
    # If player one won
    if player_one_score - player_two_score > 0.0001:
        match_history[match_num] = {
            'Player_1': player_one,
            'Player_2': player_two,
            'Score': str(player_one_score) + "-" + str(player_two_score),
            'Result': "W"
        }
    # If player two won
    elif player_two_score - player_one_score > 0.0001:
        match_history[match_num] = {
            'Player_1': player_two,
            'Player_2': player_one,
            'Score': str(player_two_score) + "-" + str(player_one_score),
            'Result': "W"
        }
    
    # If draw
    else:
        match_history[match_num] = {
            'Player_1': player_one,
            'Player_2': player_two,
            'Score': str(player_one_score) + "-" + str(player_two_score),
            'Result': "D"
        }
        
def create_standings_frame(standings_df):
    """
    Creates a standings frame for a given standings_d
    """
    
    
    global window
    global standings_frame
    ###################################################################################
    ## STANDINGS DATAFRAME
    ###################################################################################
    # The show='headings' hides the first column (column #0) of the Treeview.
    standings_frame = tk.Frame(window, relief = tk.SUNKEN, borderwidth = 3)
    standings_frame.grid(row = 1, column = 2, sticky = "nsew")

    standings_frame.columnconfigure(1, weight = 1)
    standings_frame.rowconfigure(1, weight = 1)

    tree = ttk.Treeview(standings_frame, show = 'headings')
    tree.grid(row = 1, column = 1, sticky = "nsew")

    # Add the columns to the tree.columns attribute. <- this needs to be an iterable.
    tree["columns"] = list(standings_df.columns)

    for header in list(standings_df.columns):
        # This sets up the column itself
        # tree.column basically does things to the column name specified in tree["columns"]
        tree.column(header, width = 100, anchor = "w")
        # This sets up the header bar
        tree.heading(header, 
                     text = header, 
                     anchor = "w", 
                     command = lambda _col = header: treeview_sort_column(tree, _col, False))

    for index, row in standings_df.iterrows():
        # Note that the second argument is where the thing is inserted. 0 inserts it at the
        # top. In other words, the last entry of the list will be at the top
        # tree.insert("", 0, text = index, values = list(row))

        # Using "end" inserts the thingy at the end instead.
        tree.insert("", "end", text = index, values = list(row))

        
def create_reporting_frame(pairings_list):
    global window
    global reporting_frame
    global tourney_num
    global report_slips
    global var2
    global drop_entry
    ###################################################################################
    ## REPORTING FRAME
    ###################################################################################

    reporting_frame = tk.Frame(window, borderwidth = 3)
    reporting_frame.grid(row = 1, column = 1, sticky = "nsew")

    # The player scores appear at the bottom because the computer is responding to the 
    # height of the app
    reporting_frame.rowconfigure(0, weight = 1)
    reporting_frame.columnconfigure(0, weight = 1)
    reporting_frame.columnconfigure([1,5], minsize = 100)

    # This is where we will keep our list of spinbox instances 
    # We reset this whenever reporting_frame resets
    report_slips = []
    
    # Keep copy the pairings_list so that we don't make changes to the original 
    # list - need to do this for "undo results" options
    pairings_list_copy = copy.deepcopy(pairings_list)
    
    for pairing in pairings_list_copy:
        if "Bye" in pairing:
            label = tk.Label(reporting_frame, text = f"{pairing[0]} has a bye.")
            label.grid(row = pairings_list_copy.index(pairing) + 1, column = 3, columnspan = 2, pady = 20)
            
            # Don't want to add result here - messes with match history
            # result should only be submitted during submit results.
            # add_result(tourney_num, pairing[0], pairing[1], 1, 0, match_history)
            # tourney_num += 1

        else:
            label_1 = tk.Label(reporting_frame, text = f"{pairing[0]}")

            # Create a spinbox for the scoring
            # We can use .get() to get the value of the spinval
            score_1 = ttk.Spinbox(reporting_frame, from_ = 0, to = 100)
            score_2 = ttk.Spinbox(reporting_frame, from_ = 0, to = 100)

            # Append these instances as a pairing
            report_slips.append([score_1, score_2])


            label_2 = tk.Label(reporting_frame, text = f"{pairing[1]}")
            
            row_index = pairings_list_copy.index(pairing) + 1
            
            label_1.grid(row = row_index, column = 1, columnspan = 2, sticky = "e")
            score_1.grid(row = row_index, column = 3, padx = 15, pady = 5)
            score_2.grid(row = row_index, column = 4, padx = 15)
            label_2.grid(row = row_index, column = 5, columnspan = 2, sticky = "w")

    submit = tk.Button(reporting_frame, text = "Submit Results", command = submit_results)
    submit.grid(row = len(pairings_list_copy) + 2, column = 3, columnspan = 2, pady = 5, sticky = "ew")
    
    # Create an undo button. It should be greyed out if num_rounds == 1, i.e. if its the
    # first round of the tournament.
    if num_rounds == 1:
        undo_button = tk.Button(reporting_frame, text = "Undo Previous Result", state = tk.DISABLED)
    else:
        undo_button = tk.Button(reporting_frame, text = "Undo Previous Result", command = undo_results)
        
    undo_button.grid(row = len(pairings_list_copy) + 3, column = 3, columnspan = 2, pady = 5, sticky = "ew")
    
    # Testing this - saves current match history. Should be disabled on first round
    if num_rounds == 1:
        save_history = tk.Button(reporting_frame, text = "Save Current Match History", state = tk.DISABLED)
    else:
        save_history = tk.Button(reporting_frame, text = "Save Current Match History", command = save_match_history)

    save_history.grid(row = len(pairings_list_copy) + 4, column = 3, columnspan = 2, pady = 5, sticky = "ew")
    
    # Create widgets for dropping players from the tournament.
    drop_entry = tk.Entry(reporting_frame)
    drop_button = tk.Button(reporting_frame, text = "Drop Player (One name per click)", command = drop_player)
    drop_entry.grid(row = len(pairings_list_copy) + 5, column = 3, pady = 5)
    drop_button.grid(row = len(pairings_list_copy) + 5, column = 4, pady = 5, padx = 20)
    
    # This provides users with the option to end the tournament immediately.
    var2 = tk.IntVar()
    end_tournament = tk.Checkbutton(reporting_frame, 
                                    text = "End the Tournament upon submission of results",
                                    variable = var2)
    end_tournament.grid(row = len(pairings_list_copy) + 7, column = 3, columnspan = 2, pady = 5)
    
    # This buffers the BOTTOM of the screen. Very nice.
    reporting_frame.rowconfigure(len(pairings_list_copy) + 8, weight = 1)
    
    


In [5]:
# This runs only once - this is the basically the initializations step
# Edit this for save and quit functionality.

# if continue_tourney is not None

def initialize_tournament(list_of_players, previous_match_history = None):
    """
    Begin the tournament. Note that the user may have provided a 
    different set of match histories to use, in which case we'll need to load their match
    histories and use those instead.
    """
    
    if previous_match_history is not None:
        # Create the standings from the previous match history
        standings_df = update_standings(previous_match_history, list_of_players)
        
        # Make match history a completely new copy of the previous match history.
        # This preserves previous match history
        match_history = copy.deepcopy(previous_match_history)
        
        # Adjust the tourney_num (i.e., the match_history key) to previous number + 1
        tourney_num = max(previous_match_history.keys()) + 1
    
    # If no previous match_history provided, initialize normally.
    else:
        standings_df = pd.DataFrame(list(zip(list_of_players, 
                                     [0 for i in list_of_players],
                                     [0 for i in list_of_players],
                                     [0 for i in list_of_players])), 
                                    columns = ["Player", "Played", "Points", "OMW"])

        match_history = {}
        tourney_num = 0
    
    return standings_df, match_history, tourney_num

standings_df, match_history, tourney_num = initialize_tournament(list_of_players, 
                                                                 previous_match_history = previous_match_history)

# These are always initialized as these values, and thus can be out of the function
tourney_history = {}
pairings_history = {}
num_rounds = 1

pairings_list = determine_pairings(standings_df, match_history, seeding = var1.get())

# pairings_list

['James', 'Dash', 'Prism', 'Lexi', 'Dorinthea', 'Bravo', 'Kano', 'Ira']
matching James
[]

['Prism', 'Lexi', 'Dorinthea', 'Bravo', 'Kano', 'Ira']
matching Prism
[]

['Dorinthea', 'Bravo', 'Kano', 'Ira']
matching Dorinthea
[]

['Kano', 'Ira']
matching Kano
[]



In [6]:
def submit_results():
    """
    This function does a LOT. 
    
    It runs an entire swiss round, updates standings, prepares new pairings, then finally
    replaces the frames in the tk Window with new ones.
    """
    
    global tourney_num
    global pairings_list
    global reporting_frame
    global standings_frame
    global report_slips
    global standings_df
    global window
    global num_rounds
    global var2
    
    # Save the current match_history into dataframes with specific keys
    # Need to deepcopy to prevent overwriting <- yup, that was causing the issue
    match_history_copy = copy.deepcopy(match_history)
    tourney_history[num_rounds] = match_history_copy
    
    # Error handling - if there is a non-float in the spinboxes, we indicate where the
    # error is
    for result in report_slips:

        try:
            score_one = float(result[0].get())
            score_two = float(result[1].get())
        except:
            # If exception, don't let the function continue to run
            error_label = tk.Label(reporting_frame, text = f"Error: Match {report_slips.index(result) + 1} has a non-number score.", fg = "red")
            error_label.grid(row = len(pairings_list) + 1, column = 3, columnspan = 2, pady = 5)
            return
    
    # If there was a bye, then pairings_list is gonna have an extra element.
    # Get rid of it
    pairings_list_copy = copy.deepcopy(pairings_list)
    
    if len(pairings_list) != len(report_slips):
        # Report the bye result here <- this should trigger ONLY if there was
        # a bye
        add_result(tourney_num, 
                   pairings_list_copy[0][0], 
                   pairings_list_copy[0][1], 1, 0, match_history)
        tourney_num += 1
        pairings_list_copy.pop(0)
    
    # Actual meat of the problem
    for index, result in enumerate(report_slips):
        score_one = float(result[0].get())
        score_two = float(result[1].get())
        
        add_result(tourney_num, 
                   pairings_list_copy[index][0], pairings_list_copy[index][1], 
                   score_one, score_two, 
                   match_history)
        tourney_num += 1
    
    # At this point, calculate new standings and pairings
    
    standings_df = update_standings(match_history, 
                                    list_of_players)
    
    # Save the current pairings_list into the dictionary
    pairings_history[num_rounds] = pairings_list
    
    # Regardless of whether the tourney ends, display final standings.
    reporting_frame.destroy()
    standings_frame.destroy()
    
    # Display current standings df
    create_standings_frame(standings_df)
    
    # Check if tournament has ended
    end_tourney = var2.get()
    
    # If odd number of players:
    if len(standings_df["Player"].unique()) % 2 != 0:
        # Automatically end tourney if num_rounds would be equal to num players
        if num_rounds >= len(standings_df["Player"].unique()):
            end_tourney = True
    else:
        # Automatically end tourney at num_rounds = num_players - 1
        if num_rounds >= len(standings_df["Player"].unique()) - 1:
            end_tourney = True
    
    # print(num_rounds, end_tourney)
    num_rounds += 1
    
    # If tournament has ended, create special frame <- possibly create function for this
    if end_tourney:
        reporting_frame = tk.Frame(window, borderwidth = 3)
        reporting_frame.grid(row = 1, column = 1, sticky = "nsew")

        # The player scores appear at the bottom because the computer is responding to the 
        # height of the app
        reporting_frame.rowconfigure(0, weight = 1)
        reporting_frame.columnconfigure(0, weight = 1)
        
        
        
        info_frame = tk.Frame(reporting_frame)
        info_frame.grid(row = 1, column = 1, sticky = "nsew")
        
        info_label = tk.Label(info_frame, text = "The Tournament has ended.\n\nThe final standings are presented on the right.\nYou can save the match history using\nthe button on the right.\n\nThanks for playing!")
        info_label.grid(row = 0, column = 0, sticky = "nsew", padx = 50)
        
        # Add extras to the End Screen
        extras_frame = tk.Frame(reporting_frame)
        extras_frame.grid(row = 2, column = 1, sticky = "nsew")
        
        # Save Match History
        save_history = tk.Button(info_frame, text = "Save Current Match History", command = save_match_history)
        save_history.grid(row = 0, column = 1, pady = 20, sticky = "ew", padx = 25)
        
        # Track ELO of players
        elo_label = tk.Label(extras_frame, text = "Track the ELO of your players!\n\n"
                             "Create a new ELO tracking file or load a CSV file\ncontaining"
                             " pre-existing ELO records. Must contain\na 'Player' and 'ELO' column."
                             "\n\nNote that loading an ELO file will ask you to\nload and save a file"
                             "in quick succession.")
        create_elo_button = tk.Button(extras_frame, text = "Create ELO file", command = create_new_elo)
        load_elo_button = tk.Button(extras_frame, text = "Load ELO file", command = load_elo_csv)
        elo_label.grid(row = 2, column = 0, pady = 50, padx = 20)
        create_elo_button.grid(row = 2, column = 1, pady = 10, padx = 20, sticky = "ew")
        load_elo_button.grid(row = 2, column = 2, pady = 10, padx = 20, sticky = "ew")
        
        # This buffers the BOTTOM of the screen. Very nice.
        # Arbitrarily large row number
        reporting_frame.rowconfigure(10, weight = 1)
        
        return
        
    else:    
        pairings_list = determine_pairings(standings_df, match_history)
    
        create_reporting_frame(pairings_list)
        
def undo_results():
    """
    Undo previous results button.
    
    Testing complete - may still be some bugs, but overall should be OKAY.
    """
    # The current round number hasn't been saved yet
    global num_rounds
    global standings_frame
    global reporting_frame
    global match_history
    global tourney_num
    global pairings_list
    global list_of_players
    
    # Cut the round number.
    num_rounds -= 1
    # tourney_num -= len(pairings_history[num_rounds])
    
#     # First thing first, it undoes the currently saved tourney_history and pairings_history
#     # Note that this DELETES this data
#     # May not need to pop because the latest stuff hasn't actually been saved yet.
#     tourney_history.pop(num_rounds)
#     pairings_history.pop(num_rounds)
    
    # Destroy the current reporting and standing frames.
    reporting_frame.destroy()
    standings_frame.destroy()
    
    # Create reporting frame from the previous pairings_list
    # This works now
    pairings_list = pairings_history[num_rounds]
    create_reporting_frame(pairings_list)
    
    # Recreate the previous round's standing_df from match_history
    # If last round was the very first round, recreate the starting dataframe from
    # list of players.
    if num_rounds == 1:
        temp_df = pd.DataFrame.from_dict(previous_match_history).transpose()
        list_of_players = list(set(temp_df["Player_1"]).union(set(temp_df["Player_2"])))
        
        if "Bye" in list_of_players:
            list_of_players.pop(list_of_players.index("Bye"))
        
        standings_df, \
        match_history, \
        tourney_num = initialize_tournament(list_of_players,
                                            previous_match_history = previous_match_history)

        # May not need the following since we can now initialize_tournament using the 
        # above function.
#         standings_df = pd.DataFrame(list(zip(list_of_players, 
#                              [0 for i in list_of_players],
#                              [0 for i in list_of_players],
#                              [0 for i in list_of_players])), 
#                             columns = ["Player", "Played", "Points", "OMW"])
        
#         # Note that we also need to reset match_history, otherwise it will keep adding on to 
#         # the now-obsolete results
#         match_history = {}
        
    else:
        # Right, hold up - gonna do some WEIRD SHIT
        #last_round_dict = dict_symmetric_difference(tourney_history[num_rounds], 
        #                                            tourney_history[num_rounds-1])
        
        # Get the list of players in the previous round
        # Need to do this if you want to have functionality to drop players.
        # temp_df = pd.DataFrame.from_dict(tourney_history[num_rounds]).transpose()
        # temp_df = pd.DataFrame.from_dict(last_round_dict).transpose()
        # list_of_players = list(set(temp_df["Player_1"]).union(set(temp_df["Player_2"])))
        
        list_of_players = list(np.array(pairings_list).flatten())
        
        if "Bye" in list_of_players:
            list_of_players.pop(list_of_players.index("Bye"))
        
        standings_df = update_standings(tourney_history[num_rounds], list_of_players)
        # Note that we also need to reset match_history, otherwise it will keep adding on to 
        # the now-obsolete results
        match_history = tourney_history[num_rounds]
        tourney_num -= len(pairings_history[num_rounds])
    
    # Display current standings df
    create_standings_frame(standings_df)
    

    
def save_match_history():
    """
    Saves the current match history. Does NOT save pairings.
    """
    # Opens a dialogue that asks to save the file
    filepath = asksaveasfilename(
        defaultextension="csv",
        filetypes=[("CSV Files", "*.csv"), ("All Files", "*.*")],
    )
    
    # If nothing was entered, do nothing
    if not filepath:
        return
    
    # Save the current match history in the file directory.
    match_history_df = pd.DataFrame.from_dict(match_history).transpose()
    
    match_history_df.to_csv(filepath)

def drop_player():
    """
    Drops the player name in the "drop_entry" Entry.
    
    Weird quirk. Current behaviour is that you have to re-drop all dropped players if you
    undo a result.
    
    Generates errors as necessary.
    """
    global drop_entry
    global list_of_players
    global reporting_frame
    global drop_error_label
    
    if drop_error_label is not None:
        drop_error_label.destroy()
    
    dropped_player = drop_entry.get()
    
    # print(dropped_player)
    
    # Attempt to remove the player from the list of players
    if dropped_player in list_of_players:
        # Drop player
        list_of_players.pop(list_of_players.index(dropped_player))
        
        # Print confirmation message.
        drop_error_label = tk.Label(reporting_frame, text = f"{dropped_player} successfully dropped.", fg = "red")
        drop_error_label.grid(row = len(pairings_list) + 6, column = 3, columnspan = 2, pady = 5)
        return
        
    else:
        # Inform user that player was not found.
        drop_error_label = tk.Label(reporting_frame, text = f"Error: {dropped_player} not found in tournament. Please check spelling.", fg = "red")
        drop_error_label.grid(row = len(pairings_list) + 6, column = 3, columnspan = 2, pady = 5)
        return
    

In [7]:
def expected_score(player_rating, opponent_rating):
    """
    What is a player's expected score considering their own vs. their opponent's rating.
    """
    QA = 10**(player_rating / 400)
    QB = 10**(opponent_rating / 400)
    return QA/(QA+QB)

def update_elo(player_rating, expected_score, actual_score, k_factor = 20):
    """
    Recalculates ELO given the results of a tournament.
    
    Actual/Expected scores must be the sum of the results from the tournament
    """
    new_rating = player_rating + k_factor * (actual_score - expected_score)
    return new_rating

def load_elo_csv():
    """
    Loads and makes changes to the elo_csv.
    """
    
    filepath = askopenfilename(
        filetypes=[("CSV Files", "*.csv"), ("All Files", "*.*")]
    )
    
    # If no file selected, do nothing
    if not filepath:
        return
    
    # Load initial elo
    current_elo_df = pd.read_csv(filepath)
    
    # Dictionary of players initial ELOs
    player_initial_elo = dict(zip(current_elo_df.Player, current_elo_df.ELO))
    
    # Find new players
    new_players = list(set(list_of_players).symmetric_difference(set(player_initial_elo.keys())))
    
    # If there are new players, initialize them at 1500 ELO
    for player in new_players:
        player_initial_elo[player] = 1500
        
    player_expected_scores = {name: [] for name in player_initial_elo.keys()}
    
    # Go through each match to get expected scores
    for match in match_history:
        # If there is a bye, ignore it completely
        if match_history[match]["Player_2"] == "Bye":
            continue

        player_one = match_history[match]["Player_1"]
        player_two = match_history[match]["Player_2"]

        # Calculate expected scores
        player_expected_scores[player_one].append(expected_score(player_initial_elo[player_one],
                                                                 player_initial_elo[player_two]))
        player_expected_scores[player_two].append(expected_score(player_initial_elo[player_two],
                                                                 player_initial_elo[player_one]))
        
    # Create a dictionary containing each player's new ELO
    # Round each player's ELO to a whole number.
    player_new_elo = {name: np.round(update_elo(player_initial_elo[name], 
                                   sum(player_expected_scores[name]), 
                                   standings_df.loc[standings_df["Player"] == name,"Points"].values[0], 
                                   k_factor = 20)) for name in player_initial_elo}
    
    # Convert to dataframe
    new_elo_df = pd.DataFrame.from_dict(player_new_elo, orient = "index").reset_index()
    new_elo_df.columns = ["Player", "ELO"]
    new_elo_df.sort_values("ELO", ascending = False, inplace = True)
    
    filepath = asksaveasfilename(
        defaultextension="csv",
        filetypes=[("CSV Files", "*.csv"), ("All Files", "*.*")]
    )
    
    # If no file selected, do nothing
    if not filepath:
        return
    
    print(filepath)
    new_elo_df.to_csv(filepath)
    
def create_new_elo():
    """
    Creates a new ELO CSV.
    
    Mostly the same as the other one, but without loading in.
    """
    # If there are new players, initialize them at 1500 ELO
    player_initial_elo = {name:1500 for name in list_of_players}
    
    player_expected_scores = {name: [] for name in player_initial_elo.keys()}
    
    # Go through each match to get expected scores
    for match in match_history:
        # If there is a bye, ignore it completely
        if match_history[match]["Player_2"] == "Bye":
            continue

        player_one = match_history[match]["Player_1"]
        player_two = match_history[match]["Player_2"]

        # Calculate expected scores
        player_expected_scores[player_one].append(expected_score(player_initial_elo[player_one],
                                                                 player_initial_elo[player_two]))
        player_expected_scores[player_two].append(expected_score(player_initial_elo[player_two],
                                                                 player_initial_elo[player_one]))
        
    # Create a dictionary containing each player's new ELO
    # Round each player's ELO to a whole number.
    player_new_elo = {name: np.round(update_elo(player_initial_elo[name], 
                                   sum(player_expected_scores[name]), 
                                   standings_df.loc[standings_df["Player"] == name,"Points"].values[0], 
                                   k_factor = 20)) for name in player_initial_elo}
    
    # Convert to dataframe
    new_elo_df = pd.DataFrame.from_dict(player_new_elo, orient = "index").reset_index()
    new_elo_df.columns = ["Player", "ELO"]
    new_elo_df.sort_values("ELO", ascending = False, inplace = True)
    
    filepath = asksaveasfilename(
        defaultextension="csv",
        filetypes=[("CSV Files", "*.csv"), ("All Files", "*.*")]
    )
    
    # If no file selected, do nothing
    if not filepath:
        return
    
    new_elo_df.to_csv(filepath)

In [9]:
# This is our testing ground

# Function that sorts treeview columns
# https://stackoverflow.com/questions/1966929/tk-treeview-column-sort
# WORKS!
def treeview_sort_column(tv, col, reverse):
    l = [(tv.set(k, col), k) for k in tv.get_children('')]
    l.sort(reverse=reverse)

    # rearrange items in sorted positions
    for index, (val, k) in enumerate(l):
        tv.move(k, '', index)

    # reverse sort next time
    tv.heading(col, text=col, command = lambda _col = col: treeview_sort_column(tv, 
                                                                            _col,
                                                                            not reverse))

standings_frame = None
report_slips = None
drop_entry = None
drop_error_label = None
var2 = None

window = tk.Tk()
window.title("Swiss Tournament Manager")
# window.geometry("300x400")

# One only row, so the frames adjust to height of window
window.rowconfigure(1, minsize = 300, weight = 1)

# On column 2, so the Standings Table adjusts rather than the player reporting - perhaps
# both should?
window.columnconfigure(2, minsize = 400, weight = 1)

# Create the reporting frame using a given standings_df and pairings_list
create_reporting_frame(pairings_list)

# Create the standings frame using a given standings_df
create_standings_frame(standings_df)

window.mainloop()